Experiment settings

In [16]:
import simpy
import numpy as np
import pandas as pd
import random
import math
RANDOM_SEED = 42
### problem paramters ####
order_time_max = 5
order_time_min = 1
food_prepare_min = 2
food_prepare_max = 6

n_kitchen = 2
n_counter = 2
n_customer = 150
SIM_TIME = 12 * 60 #12 hours

random.seed(RANDOM_SEED)

# helper functions, monitoring and pandas table


In [17]:
def round_down(n, decimals=1):
    multiplier = 10 ** decimals
    return abs(math.floor(n*multiplier + 0.5) / multiplier)

counter_total_wait_times = 0
env = simpy.Environment()
# ,"Clock Time","Interarrival Time","Service Time","Wait Time","Queue Length","Total Time In System"
column_names = ["Arrival Time","Service Start Time","Queue time","Exit Time","Total Wait Time","Total Service Time","Total Time in System"]
result_fifo = np.zeros((n_customer,len(column_names)))
result_tick = np.zeros((n_customer,len(column_names)))


# Simulation setting
## A) FIFO Algorithim

In [18]:
class Counter(object):
#     Counters to take order
    def __init__(self,env,num_counter):
        self.env = env
        self.counter = simpy.Resource(env,num_counter)
        self.counter_waiting = 0
        self.service_start = None
        
    def take_order(self,cus,env):
        self.service_start = env.now
        print("%s is placing order at counter %.2f" %(cus,self.service_start))
        time_taken_to_place_order = np.random.uniform(order_time_min, order_time_max)
        yield self.env.timeout(time_taken_to_place_order)
        print("Order of %s sent to kitchen at %.2f" %(cus, env.now))
        # yield env.process(tank_truck(env, fuel_pump))

    def receive_order(self,cus,env,resource):
        global counter_total_wait_times

        with resource.kitchen.request() as my_turn:
            yield my_turn
            yield env.process(resource.prepare_food(cus,env))
            service_end = env.now
            print("%s collected the food at %.2f" %(cus, service_end))
            # Record idle counter and add to total count
            # counter_total_wait_times += (service_end-self.service_start)

class Kitchen(object):
    # Kitchen to prepare food
    def __init__(self,env,num_kitchen):
        self.env = env
        self.kitchen = simpy.Resource(env,num_kitchen)
    
    def prepare_food(self,cus,env):
        print("Kitchen is preparing food for %s at %.2f" %(cus, env.now))
        food_prepare_time = np.random.uniform(food_prepare_min,food_prepare_max)
        yield self.env.timeout(food_prepare_time)
        print("Cooked food for %s at %.2f" %(cus, env.now))

def customer(env, label, queue, kitchen, data):
#     the customer process arrive at the restaurant and request counter to take order
    label = label-1
    arrive_time = env.now
    print("%s entering the queue at %.2f"%(label,arrive_time))
#     data[label,0]=label
    data[label,0]= round_down(arrive_time)
    with queue.counter.request() as my_turn:
        yield my_turn
        service_start = env.now
        data[label,1] = round_down(service_start)
        queue_time = service_start - arrive_time
        data[label,2]= round_down(queue_time)
        # placing order at counter
        yield env.process(queue.take_order(label,env))
        # waiting order at counter
        prepare_food_start = env.now
        # counter is idle now
    yield env.process(queue.receive_order(label,env,kitchen))
    # prepare_food_end = round_down(env.now)
    # counter_total_wait_times += round_down(prepare_food_end - prepare_food_start)
    # receive food from counter
    prepare_food_end = env.now
    data[label,3] = round_down(prepare_food_end)

    kitchen_prepare_duration = (prepare_food_end-prepare_food_start)
    # total wait time
    data[label,4] = round_down(kitchen_prepare_duration+queue_time)
    # total service time
    data[label,5] = round_down(prepare_food_end-service_start)
    # total time in system
    data[label,6] = round_down(prepare_food_end-arrive_time)


## Start Ticket Simulation

In [19]:
# Simlating possion process for customer arrival
def customer_arrivals(env, num_counter,num_kitchen):
    """Create new *customer* until the sim time reaches 120. with poisson process"""
    counter = Counter(env,num_counter)
    kitchen = Kitchen(env,num_kitchen)
    for i in range(n_customer):
        yield env.timeout(random.expovariate(1 / 5))
        env.process(customer(env, i+1, counter, kitchen, result_fifo))

env.process(customer_arrivals(env, n_counter,n_kitchen))
env.run(until=SIM_TIME)

d food for 48 at 213.48
48 collected the food at 213.48
Kitchen is preparing food for 51 at 213.48
Order of 52 sent to kitchen at 215.07
Cooked food for 50 at 216.94
50 collected the food at 216.94
Kitchen is preparing food for 52 at 216.94
Cooked food for 51 at 217.40
51 collected the food at 217.40
Cooked food for 52 at 220.38
52 collected the food at 220.38
53 entering the queue at 224.37
53 is placing order at counter 224.37
Order of 53 sent to kitchen at 226.55
Kitchen is preparing food for 53 at 226.55
54 entering the queue at 229.59
54 is placing order at counter 229.59
Cooked food for 53 at 230.69
53 collected the food at 230.69
Order of 54 sent to kitchen at 231.98
Kitchen is preparing food for 54 at 231.98
55 entering the queue at 234.29
55 is placing order at counter 234.29
56 entering the queue at 235.22
56 is placing order at counter 235.22
Cooked food for 54 at 237.07
54 collected the food at 237.07
Order of 55 sent to kitchen at 237.61
Kitchen is preparing food for 55 at

## FIFO Result & Analysis

In [20]:

# Converting to pandas
np_arr = np.array(result_fifo).reshape(n_customer,-1)
labels = [*range(1,n_customer+1)]
df_fifo=pd.DataFrame(data = np_arr,index=labels,columns=column_names)

total_wait_time = np.sum(df_fifo["Total Wait Time"])
total_time_in_system = np.sum(df_fifo["Total Time in System"])
total_service_time = np.sum(df_fifo['Total Service Time'])
print("")

def display(i):
    print()
    print("Counter Total Idle Time => %.2f Minutes" % counter_total_wait_times)
    print()
    print("Average Waiting Time => %.2f Minutes" % (total_wait_time / i))
    print("Average Service Time => %.2f Minutes" % (total_service_time / i))
    print("Average Time Spent In System => %.2f Minutes" % (total_time_in_system / i))

display(n_customer)



Counter Total Idle Time => 0.00 Minutes

Average Waiting Time => 4.17 Minutes
Average Service Time => 6.74 Minutes
Average Time Spent In System => 6.90 Minutes
